# Sumary
**Corporacion Favorita** has the highest incomes in Ecuador for selling grocery, clothes, toys, … by retail. Its large stores are located in the highland region specially in Pichincha state because Corporacion Rosado, the fourth company in incomes, controls the market in the coast region. For this reason, the selected data set has been delimited for Pichincha state in order to forecaste the time serie using regression trees.

# Load libraries 

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error



# Load data

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
train = pd.read_csv("../input/train.csv", parse_dates=['date'])
#test=pd.read_csv("../input/test.csv",parse_dates=['date'])
stores = pd.read_csv("../input/stores.csv") 

# Data processing
 Subseting, join and aggregating  data

In [ ]:
t=train.groupby([ 'store_nbr','date'], as_index=False).agg({"unit_sales": "sum"})
train = pd.merge(t, stores, how='left', on=['store_nbr'])
mask=train['state']=='Pichincha'
train=train.loc[mask]
train=train.groupby(['date'], as_index=False).agg({"unit_sales": "sum"})


In [ ]:
train.head(10)

In [ ]:
#train1.groupby('month', as_index=False).agg({"unit_sales": "sum"})

#MUPI_COM["valor"].plot()

## Sales for Pichincha state (train data)

In [ ]:
train.tail(12)

In [ ]:

plt.plot(train['unit_sales'])
plt.show()

There is a trend over time especially for the 2015 year, by 2015 and 2016 the slope gets lower and loses trend. It’s easy to recognize how the sales increasing over the last quarters.

# Sample split for train and test data

In [ ]:
numpy.random.seed(123)

In [ ]:
train_size = int(len(train) * 0.75)
test_size = len(train) - train_size

print(train_size,test_size, len(train))

In [ ]:
train1= train[0:train_size]
test =  train[train_size:len(train)]
print(len(train1), len(test))

In [ ]:
train1=train1.set_index("date")
test=test.set_index("date")
train=train.set_index("date")
train1=train1.values
test=test.values
train=train.values

Determine the number of previous time steps to use as input variables to predict the next time period. 
    In this case (look_back) determinated to 1

In [ ]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)


In [ ]:

look_back = 1
trainX, trainY = create_dataset(train1, look_back)
testX, testY = create_dataset(test, look_back)

# Multilayer Perceptron model

A simple network with 1 input (look_back)  , 1 hidden layer with 8 neurons and one (1) output layer.

In [ ]:

model = Sequential()
model.add(Dense(8, input_dim=look_back, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=200, batch_size=2, verbose=2)

 # Performance of the model

In [ ]:
trainScore = model.evaluate(trainX, trainY, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(testX, testY, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
 
trainPredictPlot = numpy.empty_like(train)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
 
testPredictPlot = numpy.empty_like(train)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(train)-1, :] = testPredict
 
plt.plot(train)
plt.show()
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()
plt.plot(train)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()



The average error in the training data is 86.486 units and the averrage in the test data is 114.344 units sold per day